# Service Recovery and Ridership Recovery Analysis 

based on a baseline date and current GTFS, did coverage return equitably or not. 

In [99]:
import partridge as ptg
import pandas as pd
import altair as alt

In [4]:
latest_gtfs = '2021-04-11-latest'
baseline_gtfs = '2019-Dec-28-00:27:34-bcfeaef'

In [10]:
bus_ridership = pd.read_parquet("metro-bus-covid-recovery.parquet")

In [15]:
date, service_ids = ptg.read_busiest_date(baseline_gtfs)

In [24]:
def load_gtfs_into_df(data_dir):
    """
    Loads a GTFS file into a merged dataframe 
    from the stops , stop times, routes, and 
    trips table
    """
    data_dir = f"./{data_dir}/"
    stops = pd.read_csv(f'{data_dir}stops.txt')
    stop_times = pd.read_csv(f'{data_dir}stop_times.txt')
    routes = pd.read_csv(f'{data_dir}routes.txt')
    trips = pd.read_csv(f'{data_dir}trips.txt')
    merged = pd.merge(routes,
            pd.merge(trips,
                pd.merge(stop_times, 
                         stops, 
                         on='stop_id',
                         how = 'left',
                         validate='m:1'
                    ),
                )
            )
    return merged

In [25]:
def calculate_number_of_trips(df):
    """
    For a merged DF, calaculate the number of 
    
    weekday trips, sat trips, and sunday trips 
    """
    weekday = df[df.service_id.str.endswith('Weekday')]
    saturday = df[df.service_id.str.endswith('Saturday')]
    sunday = df[df.service_id.str.endswith('Sunday')]
    return {'weekday': weekday.trip_id.nunique(),
             'saturday': saturday.trip_id.nunique(),
             'sunday': sunday.trip_id.nunique()}

How many trips per day is Metro running?

In [26]:
current_df = load_gtfs_into_df(latest_gtfs)

In [29]:
calculate_number_of_trips(current_df)

{'weekday': 11030, 'saturday': 8703, 'sunday': 8368}

In [30]:
baseline_df = load_gtfs_into_df(baseline_gtfs)

In [31]:
calculate_number_of_trips(baseline_df)

{'weekday': 13841, 'saturday': 8771, 'sunday': 8099}

As we can see, Metro is running approximately 2000 fewer trips per day on weekdays, ~60 on Saturdays and ~300 less  on Sundays. 

Let's now try breaking it down by line 

In [86]:
def trips_by_route(df):
    df = df.assign(route_name_corrected = 
                   df.route_id.apply(lambda x: x.split("-")[0]), 
                   day_of_week = df.service_id.apply(lambda x: x.split('_')[1])
                  )
    trips_by = pd.DataFrame(df.groupby(['route_name_corrected', 'day_of_week'])['trip_id'].nunique()).reset_index().rename({'trip_id': 'num_trips'}, axis =1)
    return trips_by

In [126]:
df1 = trips_by_route(current_df)
df2 = trips_by_route(baseline_df)

In [128]:
merged = pd.merge(
    df1,
    df2,
    how="inner",
    on=['day_of_week','route_name_corrected'],
    suffixes=['_current','_baseline']
)



let's look at one particular route. as we can see, the 10 lost 25 trips per day (or approx ~1 run per hour, although probably more given the distribution, between pre COVID service and after 

In [130]:
merged[merged["route_name_corrected"] == '10']

,route_name_corrected,day_of_week,num_trips_current,num_trips_baseline
0,10,Saturday,94,106
1,10,Sunday,94,94
2,10,Weekday,128,150


In [136]:
merged['percentage_loss'] = (merged.num_trips_baseline - merged.num_trips_current) / merged.num_trips_baseline * 100

In [139]:
merged[['percentage_loss', 'route_name_corrected', 'day_of_week']].sort_values('percentage_loss')

,percentage_loss,route_name_corrected,day_of_week
41,-203.846154,152,Saturday
61,-200.000000,165,Saturday
58,-139.130435,164,Saturday
55,-128.125000,163,Saturday
153,-65.346535,251,Weekday
...,...,...,...
238,47.252747,605,Weekday
266,48.333333,71,Weekday
229,50.000000,601,Weekday
313,52.631579,96,Weekday


for now, let's focus on weekday trip restoration. 

In [140]:
df = merged[merged.day_of_week == 'Weekday']

In [143]:
df.sort_values('percentage_loss', ascending=False)

,route_name_corrected,day_of_week,num_trips_current,num_trips_baseline,percent_trip_loss,percentage_loss
208,501,Weekday,42,101,58.415842,58.415842
313,96,Weekday,27,57,52.631579,52.631579
229,601,Weekday,114,228,50.000000,50.000000
266,71,Weekday,31,60,48.333333,48.333333
238,605,Weekday,48,91,47.252747,47.252747
...,...,...,...,...,...,...
227,60,Weekday,240,187,-28.342246,-28.342246
110,210,Weekday,168,112,-50.000000,-50.000000
178,28,Weekday,166,107,-55.140187,-55.140187
8,105,Weekday,193,122,-58.196721,-58.196721


In [144]:
df.to_csv('percentage_loss.csv')